<a href="https://colab.research.google.com/github/mit1280/fined-tuning/blob/main/phi_2_fine_tune_using_PEFT%2Binference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install accelerate transformers einops datasets peft bitsandbytes --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.8 MB/s eta 0:00:00


In [2]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import os

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model

PhiForCausalLM(
  (transformer): PhiModel(
    (embd): Embedding(
      (wte): Embedding(51200, 2560)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (h): ModuleList(
      (0-31): 32 x ParallelBlock(
        (ln): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (Wqkv): Linear4bit(in_features=2560, out_features=7680, bias=True)
          (out_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (inner_attn): SelfAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
          (inner_cross_attn): CrossAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
        )
        (mlp): MLP(
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
          (act): NewGELUActivation()
        )
      )

In [ ]:
from bitsandbytes.nn.modules import Linear8bitLt, Linear4bit
import torch.nn.init as init  # Importing the init module from PyTorch
import bitsandbytes as bnb
from contextlib import contextmanager

def noop (x=None, *args, **kwargs):
    "Do nothing"
    return x

@contextmanager
def no_kaiming():
    old_iku = init.kaiming_uniform_
    init.kaiming_uniform_ = noop
    try: yield
    finally: init.kaiming_uniform_ = old_iku

_old_8init = Linear8bitLt.__init__
_old_4init = Linear4bit.__init__

def _new_4init(self, input_features, output_features, bias=True,
               device=None, **kwargs):
    with no_kaiming():
        return _old_4init(self, input_features, output_features, bias=bias,
                          device=device, **kwargs)



def _new_8init(self, input_features, output_features, bias=True, has_fp16_weights=True,
              memory_efficient_backward=False, threshold=0.0, index=None, device=None):
    with no_kaiming():
        return _old_8init(self, input_features, output_features, bias=bias, has_fp16_weights=has_fp16_weights,
                          memory_efficient_backward=memory_efficient_backward, threshold=threshold, index=index, device=device)

Linear8bitLt.__init__ = _new_8init
Linear4bit.__init__ = _new_4init

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["Wqkv", "out_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 7,864,320 || all params: 2,787,548,160 || trainable%: 0.2821231974697076


In [ ]:
data = load_dataset("Universal-NER/Pile-NER-type", "default", split="train")

data_df = data.to_pandas()

In [ ]:
tokenizer.default_chat_template


No chat template is defined for this tokenizer - using a default chat template that implements the ChatML format (without BOS/EOS tokens!). If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



"{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

In [ ]:
template = tokenizer.default_chat_template
template = template.replace("role", "from").replace("content", "value")  # Change the system token
tokenizer.chat_template = template  # Set the new template
# tokenizer.push_to_hub("model_name")  # Upload your new template to the Hub!

In [ ]:
tokenizer.default_chat_template

"{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

In [ ]:
tokenizer.apply_chat_template(data_df.loc[0]['conversations'], tokenize=False)

'<|im_start|>human\nText: Q:\n\nPosition character based on enemy coordinates in lua\n\nI have written a function here which should turn my character based on enemy coordinates but it\'s not perfect because it does not always turn where I want it to and perhaps there is a better way of writing it\nlocal myPosition = {x = 350, y = 355}\nlocal enemyPosition = {x = 352, y = 354}\nlocal xValue, yValue, xDir, yDir, dir\n\nif myPosition.x > enemyPosition.x then\n    xValue = myPosition.x - enemyPosition.x\nelseif myPosition.x < enemyPosition.x then\n    xValue = myPosition.x - enemyPosition.x\nelse\n    xValue = 0\nend\n\nif myPosition.y > enemyPosition.y then\n    yValue = myPosition.y - enemyPosition.y\nelseif myPosition.y < enemyPosition.y then\n    yValue = myPosition.y - enemyPosition.y\nelse\n    yValue = 0\nend\n\nif xValue < 0 then\n    xDir = "TURN RIGHT"\nelseif xValue > 0 then\n    xDir = "TURN LEFT"\nend\n\nif yValue < 0 then\n    yDir = "TURN DOWN"\nelseif yValue > 0 then\n    y

In [ ]:
data_df['chat_conversations'] = data_df['conversations'].apply(lambda x: tokenizer.apply_chat_template(x, tokenize=False))

In [ ]:
data_df.loc[0]['chat_conversations']

'<|im_start|>human\nText: Q:\n\nPosition character based on enemy coordinates in lua\n\nI have written a function here which should turn my character based on enemy coordinates but it\'s not perfect because it does not always turn where I want it to and perhaps there is a better way of writing it\nlocal myPosition = {x = 350, y = 355}\nlocal enemyPosition = {x = 352, y = 354}\nlocal xValue, yValue, xDir, yDir, dir\n\nif myPosition.x > enemyPosition.x then\n    xValue = myPosition.x - enemyPosition.x\nelseif myPosition.x < enemyPosition.x then\n    xValue = myPosition.x - enemyPosition.x\nelse\n    xValue = 0\nend\n\nif myPosition.y > enemyPosition.y then\n    yValue = myPosition.y - enemyPosition.y\nelseif myPosition.y < enemyPosition.y then\n    yValue = myPosition.y - enemyPosition.y\nelse\n    yValue = 0\nend\n\nif xValue < 0 then\n    xDir = "TURN RIGHT"\nelseif xValue > 0 then\n    xDir = "TURN LEFT"\nend\n\nif yValue < 0 then\n    yDir = "TURN DOWN"\nelseif yValue > 0 then\n    y

In [ ]:
def tokenize(sample):
    tokenized_text =  tokenizer(sample["chat_conversations"], padding=True, truncation=True, max_length=512)
    return tokenized_text

In [ ]:
data = Dataset.from_pandas(data_df)

data

Dataset({
    features: ['id', 'conversations', 'chat_conversations'],
    num_rows: 45889
})

In [ ]:
tokenized_data = data.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=data.column_names)

tokenized_data

Tokenizing data:   0%|          | 0/45889 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 45889
})

In [ ]:
training_arguments = TrainingArguments(
        output_dir="phi-2-universal-NER",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        max_steps=10000,
        num_train_epochs=5
    )

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


KeyboardInterrupt: ignored

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub()

adapter_model.safetensors:   0%|          | 0.00/31.5M [00:00<?, ?B/s]

events.out.tfevents.1703196006.5f6c9292baec.596.0:   0%|          | 0.00/4.89k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1703198224.5f6c9292baec.19634.0:   0%|          | 0.00/6.81k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

'https://huggingface.co/Mit1208/phi-2-universal-NER/tree/main/'

In [ ]:
tokenizer.push_to_hub("Mit1208/phi-2-universal-NER")

CommitInfo(commit_url='https://huggingface.co/Mit1208/phi-2-universal-NER/commit/77da1272b99a989e300670313352b2d6e62b53f7', commit_message='Upload tokenizer', commit_description='', oid='77da1272b99a989e300670313352b2d6e62b53f7', pr_url=None, pr_revision=None, pr_num=None)

## Inference

In [3]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("Mit1208/phi-2-universal-NER")
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2",device_map="auto", trust_remote_code=True)
model = PeftModel.from_pretrained(model, "Mit1208/phi-2-universal-NER", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/31.5M [00:00<?, ?B/s]

In [19]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Mit1208/phi-2-universal-NER", trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [63]:
conversations = [ { "from": "human", "value": "Text: Mit Patel here from India"}, {"from": "gpt", "value": "I've read this text."},
                   {"from":"human", "value":"what is a name of the person in the text?"}]

In [64]:
inference_text = tokenizer.apply_chat_template(conversations, tokenize=False) + '<|im_start|>gpt:\n'
inference_text

"<|im_start|>human\nText: Mit Patel here from India<|im_end|>\n<|im_start|>gpt\nI've read this text.<|im_end|>\n<|im_start|>human\nwhat is a name of the person in the text?<|im_end|>\n<|im_start|>gpt:\n"

In [65]:
inputs = tokenizer(inference_text, return_tensors="pt", return_attention_mask=False)
inputs

{'input_ids': tensor([[   27,    91,   320,    62,  9688,    91,    29, 10734,   198,  8206,
            25, 11707, 33110,   994,   422,  3794,    27,    91,   320,    62,
           437,    91,    29,   198,    27,    91,   320,    62,  9688,    91,
            29,    70,   457,   198,    40,  1053,  1100,   428,  2420, 29847,
            91,   320,    62,   437,    91,    29,   198,    27,    91,   320,
            62,  9688,    91,    29, 10734,   198, 10919,   318,   257,  1438,
           286,   262,  1048,   287,   262,  2420,    30,    27,    91,   320,
            62,   437,    91,    29,   198,    27,    91,   320,    62,  9688,
            91,    29,    70,   457,    25,   198]])}

In [66]:
inputs = inputs.to("cuda")

In [67]:
tokenizer.encode("<|im_end|>"), tokenizer.eos_token

([27, 91, 320, 62, 437, 91, 29], '<|endoftext|>')

In [68]:
import torch
from transformers import StoppingCriteria
class EosListStoppingCriteria(StoppingCriteria):
    def __init__(self, eos_sequence = tokenizer.encode("<|im_end|>")):
        self.eos_sequence = eos_sequence

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        last_ids = input_ids[:,-len(self.eos_sequence):].tolist()
        return self.eos_sequence in last_ids
# output = model.generate(inputs["input_ids"], max_new_tokens=64, stopping_criteria = [EosListStoppingCriteria()])

In [69]:
outputs = model.generate(**inputs, max_length=512, pad_token_id= tokenizer.eos_token_id,
            stopping_criteria = [EosListStoppingCriteria()])

text = tokenizer.batch_decode(outputs)[0]

print(text)

<|im_start|>human
Text: Mit Patel here from India<|im_end|>
<|im_start|>gpt
I've read this text.<|im_end|>
<|im_start|>human
what is a name of the person in the text?<|im_end|>
<|im_start|>gpt:
["Mit Patel"]<|im_end|>


## Reference

1. https://github.com/rohan-paul/LLM-FineTuning-Large-Language-Models/blob/main/FineTuning_phi-1_5_with_PRFT_LoRA.ipynb
2. dataset is from https://universal-ner.github.io/
3. https://github.com/huggingface/transformers/issues/26959